In [27]:
# Créer une code de collecte automatisée des vidéos tendances YouTube via l’API officielle

In [5]:
!pip install google-api-python-client pandas

In [7]:
!pip install isodate


In [9]:
!pip install python-dotenv


In [13]:
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import isodate

# 1️⃣ Configurer l'API
load_dotenv()
API_KEY = os.getenv("YOUTUBE_API_KEY")
youtube = build('youtube', 'v3', developerKey=API_KEY)

# 2️⃣ Récupérer les catégories YouTube
def get_video_categories(youtube, region_code="FR"):
    request = youtube.videoCategories().list(
        part="snippet",
        regionCode=region_code
    )
    response = request.execute()

    return {item["id"]: item["snippet"]["title"] for item in response["items"]}


categories_fr = get_video_categories(youtube, "FR")

# 3️⃣ Traduction des genres en français
TRADUCTION_GENRES = {
    "Film & Animation": "Cinéma & Animation",
    "Autos & Vehicles": "Automobile",
    "Music": "Musique",
    "Pets & Animals": "Animaux",
    "Sports": "Sport & Fitness",
    "Travel & Events": "Voyage",
    "Gaming": "Jeux vidéo",
    "People & Blogs": "Lifestyle & Blogs",
    "Comedy": "Humour",
    "Entertainment": "Divertissement",
    "News & Politics": "Actualités",
    "Howto & Style": "Cuisine / Mode / Tutoriels",
    "Education": "Éducation",
    "Science & Technology": "Science & Technologie",
    "Nonprofits & Activism": "Associatif"
}

# 4️⃣ Paramètres
pays = ["FR", "US"]
max_videos_par_pays = 500
videos_par_page = 50

videos = []

# 5️⃣ Récupération des vidéos
for region in pays:
    next_page_token = None
    total_videos = 0

    while total_videos < max_videos_par_pays:
        request = youtube.videos().list(
            part="snippet,statistics,contentDetails",
            chart="mostPopular",
            regionCode=region,
            maxResults=min(videos_par_page, max_videos_par_pays - total_videos),
            pageToken=next_page_token
        )
        response = request.execute()

        for item in response.get("items", []):
            snippet = item.get("snippet", {})
            stats = item.get("statistics", {})
            content = item.get("contentDetails", {})

            # Durée
            duration = isodate.parse_duration(content.get("duration", "PT0S"))
            minutes, seconds = divmod(int(duration.total_seconds()), 60)

            # Genre
            category_id = snippet.get("categoryId")
            category_name = categories_fr.get(category_id, "Inconnu")
            category_fr = TRADUCTION_GENRES.get(category_name, category_name)

            # Chaîne / Youtubeur
            channel_name = snippet.get("channelTitle", "Inconnu")
            channel_id = snippet.get("channelId")
            channel_url = f"https://www.youtube.com/channel/{channel_id}"

            videos.append({
                "Video ID": item.get("id"),  # ✅ IDENTIFIANT UNIQUE
                "Titre": snippet.get("title", ""),
                "Description": snippet.get("description", ""),
                "Date de publication": datetime.strptime(
                    snippet.get("publishedAt"),
                    "%Y-%m-%dT%H:%M:%SZ"
                ).strftime("%d/%m/%Y %H:%M"),
                "Durée (min:sec)": f"{minutes}:{seconds:02}",
                "Vues": int(stats.get("viewCount", 0)),
                "Likes": int(stats.get("likeCount", 0)),
                "Commentaires": int(stats.get("commentCount", 0)),
                "Genre": category_fr,
                "Nom de la chaîne": channel_name,
                "Nom du youtubeur": channel_name,
                "Lien de la chaîne": channel_url,
                "Pays": region
            })

            total_videos += 1

        next_page_token = response.get("nextPageToken")
        if not next_page_token:
            break

# 6️⃣ DataFrame final
df = pd.DataFrame(videos)

# 7️⃣ Vérification
print(df.head())
print("Total vidéos :", len(df))
print(df["Genre"].value_counts())


      Video ID                                              Titre  \
0  HomRl9941e4                SCYLLA & FURAX BARBAROSSA - HAYAWAN   
1  NARdg5Qz3zY     SHELTER Bande Annonce  VF (2026) Jason Statham   
2  fQBPSDP_U9w  Je Débloque la Base French Tacos sur Steal a B...   
3  ZX9VdGvUDbU                                           BONBONNE   
4  DemDCXm6mvI      SHELTER Bande Annonce VF (2026) Jason Statham   

                                         Description Date de publication  \
0  Ecouter en streaming HAYAWAN : https://demain....    21/01/2026 11:30   
1  ✩ Les Films à VOIR ?  Ils sont ICI ► https://w...    20/01/2026 22:04   
2  🔴Code NOUVELLE MAP Stop the Brainrot : 5697 87...    21/01/2026 02:06   
3  Provided to YouTube by IDOL\n\nBONBONNE · HOUD...    20/01/2026 10:00   
4  Les Films à ne pas Manquer : \nhttps://youtube...    20/01/2026 19:48   

  Durée (min:sec)    Vues  Likes  Commentaires               Genre  \
0            4:17    4017    849           160            

In [15]:
df.head()

,Video ID,Titre,Description,Date de publication,Durée (min:sec),Vues,Likes,Commentaires,Genre,Nom de la chaîne,Nom du youtubeur,Lien de la chaîne,Pays
0,HomRl9941e4,SCYLLA & FURAX BARBAROSSA - HAYAWAN,Ecouter en streaming HAYAWAN : https://demain....,21/01/2026 11:30,4:17,4017,849,160,Musique,Scylla,Scylla,https://www.youtube.com/channel/UCz3KCKF7Fi32F...,FR
1,NARdg5Qz3zY,SHELTER Bande Annonce VF (2026) Jason Statham,✩ Les Films à VOIR ? Ils sont ICI ► https://w...,20/01/2026 22:04,2:53,49729,897,103,Cinéma & Animation,FilmsActu,FilmsActu,https://www.youtube.com/channel/UC_i8X3p8oZNai...,FR
2,fQBPSDP_U9w,Je Débloque la Base French Tacos sur Steal a B...,🔴Code NOUVELLE MAP Stop the Brainrot : 5697 87...,21/01/2026 02:06,180:19,463896,11377,346,Jeux vidéo,Unchained_Off,Unchained_Off,https://www.youtube.com/channel/UCiFyJ_EBmF2XJ...,FR
3,ZX9VdGvUDbU,BONBONNE,Provided to YouTube by IDOL\n\nBONBONNE · HOUD...,20/01/2026 10:00,2:41,3900,525,37,Musique,HOUDI - Topic,HOUDI - Topic,https://www.youtube.com/channel/UCAPGPotSzOhSW...,FR
4,DemDCXm6mvI,SHELTER Bande Annonce VF (2026) Jason Statham,Les Films à ne pas Manquer : \nhttps://youtube...,20/01/2026 19:48,2:53,34056,448,48,Cinéma & Animation,AuCiné,AuCiné,https://www.youtube.com/channel/UClz394yrcpOnP...,FR


In [17]:
df.to_csv("trend_videos_youtube_aline.csv", index=False)
